# AI Model: Gemma by GOOGLE
We are going to use gemma as our pretrained, open source LLM. This model will be loaded in using 

In [1]:
pip install -q --upgrade keras-nlp

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install ipywidgets

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install -q --upgrade keras  # Upgrade to Keras 3.

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install plotly

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install kaggle

Note: you may need to restart the kernel to use updated packages.


# Math Director Prompt:

You are the director of a mathematics solutions company;
you will take a given math problem, 
and break the task down into a list of individual steps that must be completed to solve the task;

for each math problem; you will delegate to a math solution assistant who will complete the task;

ONLY say "TERMINATE" after you have returned a solution for the problem.

In [7]:
import os
os.environ["KERAS_BACKEND"] = "jax" # you can also use tensorflow or torch
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "0.9" # avoid memory fragmentation on JAX backend.

import keras
import keras_nlp
import kagglehub
import kaggle
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
tqdm.pandas() # progress bar for pandas

import plotly.graph_objs as go
import plotly.express as px
from IPython.display import display, Markdown


class CFG:
    seed = 42
    preset = "gemma_1.1_instruct_2b_en" # name of pretrained Gemma
    sequence_length = 512 # max size of input sequence for training
    batch_size = 1 # size of the input batch in training
    epochs = 1 # number of epochs to train

keras.utils.set_random_seed(CFG.seed)

df1 = pd.read_csv("traincompiled.csv")
df2 = pd.read_csv("testcompiled.csv")
# df3 = pd.read_csv("AI Math Olympiad Prize/test.csv")
# df4 = pd.read_csv("AI Math Olympiad Prize/train.csv")

# df5 = pd.concat([df3, df4], axis=0)
# df5.head(1)

df = pd.concat([df1, df2], axis=0)
df.head(2)



,Question,Level,Type,Solution,Answer
0,My school's math club has 6 boys and 8 girls. ...,Level 2,counting_and_probability,"With no restrictions, we are merely picking 6 ...",3003
1,How many 4-letter words with at least one cons...,Level 3,counting_and_probability,First we count the number of all 4-letter word...,609


In [8]:
template = """Role:\nYou are an advanced AI system with exceptional mathematical reasoning and problem-solving capabilities, specifically designed to solve tricky math problems (whose answer is a non-negative integer) written in LaTeX format from the AI Mathematical Olympiad (AIMO) competition. Your task is to accurately analyze and solve intricate mathematical problems, demonstrating a deep understanding of mathematical concepts and a strong ability to apply logical reasoning strategies.\n\nInstruction:
1. Carefully read and comprehend the problem statement provided in the "Problem" section.
2. In the "Solution" section, provide a solution of the problem with detailed explanation of your logical reasoning process. Keep in mind that answer must be a non-negative integer number.
3. At the end, create a "Answer" section where you will state only the final numerical or algebraic answer, without any additional text or narrative.\n\nProblem:\n{problem}\n\nSolution:\n{solution}"""

df["prompt"] = df.progress_apply(lambda row: template.format(problem=row.Question,
                                                             solution=f"{row.Solution}\n\nAnswer:\n{row.Answer}"),
                                                             axis=1)
data = df.prompt.tolist()

  0%|          | 0/7929 [00:00<?, ?it/s]

## The code below is for the original problem set

In [9]:
# template = """Role:\nYou are an advanced AI system with exceptional mathematical reasoning and problem-solving capabilities, specifically designed to solve tricky math problems (whose answer is a non-negative integer) written in LaTeX format from the AI Mathematical Olympiad (AIMO) competition. Your task is to accurately analyze and solve intricate mathematical problems, demonstrating a deep understanding of mathematical concepts and a strong ability to apply logical reasoning strategies.\n\nInstruction:
# 1. Carefully read and comprehend the problem statement provided in the "Problem" section.
# 2. In the "Solution" section, provide a solution of the problem with detailed explanation of your logical reasoning process. Keep in mind that answer must be a non-negative integer number.
# 3. At the end, create a "Answer" section where you will state only the final numerical or algebraic answer, without any additional text or narrative.\n\nProblem:\n{problem}\n\nSolution:\n{solution}"""

# df["prompt"] = df.progress_apply(lambda row: template.format(problem=row.Question,
#                                                              solution=f"{row.Solution}\n\nAnswer:\n{row.Answer}"),
#                                                              axis=1)
# data = df.prompt.tolist()

In [10]:
def colorize_text(text):
    for word, color in zip(["Role", "Instruction", "Problem", "Solution", "Answer"],
                           ["blue", "yellow", "red", "cyan", "green"]):
        text = text.replace(f"{word}:", f"\n\n**<font color='{color}'>{word}:</font>**")
    return text

# Take a random sample
sample = data[12]

# Give colors to Instruction, Response and Category
sample = colorize_text(sample)

# Show sample in markdown
display(Markdown(sample))



**<font color='blue'>Role:</font>**
You are an advanced AI system with exceptional mathematical reasoning and problem-solving capabilities, specifically designed to solve tricky math problems (whose answer is a non-negative integer) written in LaTeX format from the AI Mathematical Olympiad (AIMO) competition. Your task is to accurately analyze and solve intricate mathematical problems, demonstrating a deep understanding of mathematical concepts and a strong ability to apply logical reasoning strategies.



**<font color='yellow'>Instruction:</font>**
1. Carefully read and comprehend the problem statement provided in the "Problem" section.
2. In the "Solution" section, provide a solution of the problem with detailed explanation of your logical reasoning process. Keep in mind that answer must be a non-negative integer number.
3. At the end, create a "Answer" section where you will state only the final numerical or algebraic answer, without any additional text or narrative.



**<font color='red'>Problem:</font>**
Each block on the grid shown in the Figure is 1 unit by 1 unit.  Suppose we wish to walk from $A$ to $B$ via a 7 unit path, but we have to stay on the grid -- no cutting across blocks.  How many different paths can we take?[asy]size(3cm,3cm);int w=5;int h=4;int i;for (i=0; i<h; ++i){draw((0,i) -- (w-1,i));}for (i=0; i<w; ++i){draw((i, 0)--(i,h-1));}label("B", (w-1,h-1), NE);label("A", (0,0), SW);[/asy]



**<font color='cyan'>Solution:</font>**
We know that we must take a 7 unit path.  If we look at the grid a little more carefully, we can see that our path must consist of 4 steps to the right and 3 steps up, and we can take those steps in any order.  So in order to specify a path, we must choose 3 of our 7 steps to be `up' (and the other 4 steps will thus be `right').  Hence the number of paths is $$ \binom{7}{3} = \frac{7 \times 6 \times 5}{3 \times 2 \times 1} = \boxed{35}. $$



**<font color='green'>Answer:</font>**
35

In [11]:
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset(CFG.preset)
gemma_lm.summary()

normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.


Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Tokenizer (type)                                   ┃                                             Vocab # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                   │                                             256,000 │
└────────────────────────────────────────────────────┴─────────────────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2048)        │   2,506,172,416 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     524,288,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,506,172,416 (9.34 GB)

 Trainable params: 2,506,172,416 (9.34 GB)

 Non-trainable params: 0 (0.00 B)

In [12]:
x, y, sample_weight = gemma_lm.preprocessor(data[0:2])

# Display the shape of each processed output
for k, v in x.items():
    print(k, ":", v.shape)

token_ids : (2, 8192)
padding_mask : (2, 8192)


In [13]:
# Take one sample
row = df.iloc[12]

# Generate Prompt using template
prompt = template.format(
    problem=row.Question,
    solution="",
)

# Infer
output = gemma_lm.generate(prompt, max_length=1024)

# Colorize
output = colorize_text(output)

# Display in markdown
display(Markdown(output))



**<font color='blue'>Role:</font>**
You are an advanced AI system with exceptional mathematical reasoning and problem-solving capabilities, specifically designed to solve tricky math problems (whose answer is a non-negative integer) written in LaTeX format from the AI Mathematical Olympiad (AIMO) competition. Your task is to accurately analyze and solve intricate mathematical problems, demonstrating a deep understanding of mathematical concepts and a strong ability to apply logical reasoning strategies.



**<font color='yellow'>Instruction:</font>**
1. Carefully read and comprehend the problem statement provided in the "Problem" section.
2. In the "Solution" section, provide a solution of the problem with detailed explanation of your logical reasoning process. Keep in mind that answer must be a non-negative integer number.
3. At the end, create a "Answer" section where you will state only the final numerical or algebraic answer, without any additional text or narrative.



**<font color='red'>Problem:</font>**
Each block on the grid shown in the Figure is 1 unit by 1 unit.  Suppose we wish to walk from $A$ to $B$ via a 7 unit path, but we have to stay on the grid -- no cutting across blocks.  How many different paths can we take?[asy]size(3cm,3cm);int w=5;int h=4;int i;for (i=0; i<h; ++i){draw((0,i) -- (w-1,i));}for (i=0; i<w; ++i){draw((i, 0)--(i,h-1));}label("B", (w-1,h-1), NE);label("A", (0,0), SW);[/asy]



**<font color='cyan'>Solution:</font>**
The grid contains 20 paths that can be taken from A to B.

Explanation:
We can solve this problem by applying the concept of combinations.  The formula for combinations is:

$$C(n, r) = \frac{n!}{r!(n-r)!}$$

In this case, n = 5 and r = 7, so the number of different paths is:

$$C(5, 7) = \frac{5!}{7!(5-7)!} = 20$$



**<font color='green'>Answer:</font>** 20

In [14]:
# Take one sample
row = df.iloc[32]

# Generate Prompt using template
prompt = template.format(
    problem=row.Question,
    solution=""
)

# Infer
output = gemma_lm.generate(prompt, max_length=1024)

# Colorize
output = colorize_text(output)

# Display in markdown
display(Markdown(output))



**<font color='blue'>Role:</font>**
You are an advanced AI system with exceptional mathematical reasoning and problem-solving capabilities, specifically designed to solve tricky math problems (whose answer is a non-negative integer) written in LaTeX format from the AI Mathematical Olympiad (AIMO) competition. Your task is to accurately analyze and solve intricate mathematical problems, demonstrating a deep understanding of mathematical concepts and a strong ability to apply logical reasoning strategies.



**<font color='yellow'>Instruction:</font>**
1. Carefully read and comprehend the problem statement provided in the "Problem" section.
2. In the "Solution" section, provide a solution of the problem with detailed explanation of your logical reasoning process. Keep in mind that answer must be a non-negative integer number.
3. At the end, create a "Answer" section where you will state only the final numerical or algebraic answer, without any additional text or narrative.



**<font color='red'>Problem:</font>**
Let $A$ equal the number of four digit odd numbers.  Let $B$ equal the number of four digit multiples of 5.  Find $A+B$.



**<font color='cyan'>Solution:</font>**
**Step 1: Identify the key information**

- The problem asks to find the sum of two quantities: $A$ and $B$.
- $A$ represents the number of four digit odd numbers.
- $B$ represents the number of four digit multiples of 5.

**Step 2: Formulate a mathematical equation**

$$A + B = ?$$

**Step 3: Solve for the answer**

Since the problem asks for an integer answer, we need to find a solution where $A+B$ is a non-negative integer.  The only way this can happen is if:

$$A = 1, B = 4$$

Therefore, the solution to the problem is:

$$A+B = 1+4 = 5$$

As of right now it seems like the llm is having trouble identifying what exactly the model is asking it to do, and subsequently what exact format the answer should be in. I'm thinking there should be a model to essentially read over the problem, identify the core goal, and format it to in a way that a manager can give the ok on proceeding with attempting to solve it.

So now it's Solution manager: Problem extrapolator, Mathmatician, Solution checker